In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [74]:
import pandas as pd
import numpy as np
input_texts,target_texts=[],[]
input_vocabulary=set()
output_vocabulary=set()
start_token='\t'
stop_token='\n'

chunk_size=10000
datalength=0
max_encoder_seq_length=0
max_decoder_seq_length=0

filehandle=open('data/moviedialog.csv','r')
df_iterator=pd.read_csv(filehandle,iterator=True,chunksize=chunk_size) #考虑到巨量文件的处理，本例子用不用不会产生任何影响，但是要是很巨大的文件，建议这么处理。
loop=True
while loop:
    try:
        chunk=df_iterator.get_chunk(chunk_size)
        datalength+=len(chunk)
        print("chunk :{}".format(len(chunk)))
        for input_t,target_t in zip(chunk.statement,chunk.reply):
            for char in input_t:
                if char not in input_vocabulary:
                    input_vocabulary.add(char)
                if char not in output_vocabulary:
                    output_vocabulary.add(char)
            max_encoder_seq_length=max_encoder_seq_length if max_encoder_seq_length>len(input_t) else len(input_t)
            max_decoder_seq_length=max_decoder_seq_length if max_decoder_seq_length>len(start_token+target_t+stop_token) else len(start_token+target_t+stop_token)
    except StopIteration:
        loop=False
        filehandle.seek(0)
        df_iterator=pd.read_csv(filehandle,iterator=True,chunksize=chunk_size)
        print("加载处理完毕")
output_vocabulary.add(start_token)
output_vocabulary.add(stop_token)
print(datalength)

chunk :10000
chunk :10000
chunk :10000
chunk :10000
chunk :10000
chunk :10000
chunk :4350
加载处理完毕
64350


In [41]:
print(output_vocabulary)
print(input_vocabulary)
print(max_encoder_seq_length)
print(max_decoder_seq_length)

{'l', 'b', '0', 'k', 'e', 'j', 'r', '6', '9', 'z', 'u', 'o', 'q', '?', '2', 'f', "'", 'w', '1', '8', 'c', 'x', 'h', 't', '.', ',', 'y', 'i', 'a', 'n', ';', 'd', '5', 'p', 's', 'g', ' ', ':', '!', '\n', 'm', '4', '\t', '7', '3', 'v'}
{'l', 'b', '0', 'k', 'e', 'j', 'r', '6', '9', 'z', 'u', 'o', 'q', '?', '2', 'f', "'", 'w', '1', '8', 'c', 'x', 'h', 't', '.', ',', 'y', 'i', 'a', 'n', ';', 'd', '5', 'p', 's', 'g', ' ', ':', '!', 'm', '4', '7', '3', 'v'}
100
102


In [4]:
input_vocabulary=sorted(input_vocabulary)
output_vocabulary=sorted(output_vocabulary)
input_vocab_size=len(input_vocabulary)
output_vocab_size=len(output_vocabulary)
input_token_index=dict([(char,i ) for i,char in enumerate(input_vocabulary)])
target_token_index=dict([(char,i) for i,char in enumerate(output_vocabulary)])
reverse_input_token_index=dict([(i,char) for i,char in enumerate(input_vocabulary)])
reverse_target_token_index=dict([(i,char) for i,char in enumerate(output_vocabulary)])

In [64]:
import numpy as np
df_iterator=pd.read_csv('data/moviedialog.csv',iterator=True,chunksize=chunk_size)
def data_genertorfromcsv(datasource,batchsize,input_token_index,target_token_index,max_encoder_seq_length,max_decoder_seq_length):
    encoder_input_datas=[]
    decoder_input_datas=[]
    decoder_target_datas=[]
    filehandle=open(datasource,'r')
    df_iterator=pd.read_csv(filehandle,iterator=True,chunksize=batchsize)
    while True:
        try:
            chunk=df_iterator.get_chunk(batchsize)
            for input_t,target_t in zip(chunk.statement,chunk.reply):
                encoder_input_data=np.zeros((max_encoder_seq_length,len(input_token_index)))
                decoder_input_data=np.zeros((max_decoder_seq_length,len(target_token_index)))
                decoder_target_data=np.zeros((max_decoder_seq_length,len(target_token_index)))
                for t,char in enumerate(input_t):
                    encoder_input_data[t,input_token_index[char]]=1
                target_t=start_token+target_t+stop_token
                for yt,chart in enumerate(target_t):
                    decoder_input_data[yt,target_token_index[chart]]=1
                    if yt>0:
                        decoder_target_data[yt-1,target_token_index[chart]]=1
                
                encoder_input_datas.append(encoder_input_data)
                decoder_input_datas.append(decoder_input_data)
                decoder_target_datas.append(decoder_target_data)
            encoder_input_datas=np.reshape(encoder_input_datas,(len(encoder_input_datas),max_encoder_seq_length,len(input_token_index)))
            decoder_input_datas=np.reshape(decoder_input_datas,(len(decoder_input_datas),max_decoder_seq_length,len(target_token_index)))
            decoder_target_datas=np.reshape(decoder_target_datas,(len(decoder_target_datas),max_decoder_seq_length,len(target_token_index)))
            yield [encoder_input_datas,decoder_input_datas],decoder_target_datas
            encoder_input_datas=[]
            decoder_input_datas=[]
            decoder_target_datas=[]
        except StopIteration:
            filehandle.seek(0)
            df_iterator=pd.read_csv(filehandle,iterator=True,chunksize=chunk_size)
            


In [75]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM,Dense,Input
batch_size=256
epochs=100
num_neurons=256
encoder_inputs=Input(shape=(None,input_vocab_size))
encoder=LSTM(num_neurons,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

decoder_inputs=Input(shape=(None,output_vocab_size))
decoder_lstm=LSTM(num_neurons,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(output_vocab_size,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics='acc')
# model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.1)

In [76]:
d=data_genertorfromcsv('data/moviedialog.csv',batchsize=batch_size,input_token_index=input_token_index,target_token_index=target_token_index,max_encoder_seq_length=max_encoder_seq_length,max_decoder_seq_length=max_decoder_seq_length)

In [77]:
import math
model.fit(d,epochs=epochs,steps_per_epoch=math.ceil(datalength/batch_size)) #利用ｍａｔｈ函数保证训练文本可以使用完，以免最后几条被丢弃，这里出去的时候没有创建验证数据生成器，格式是一样的，可以提前分好，加载的时候也是加载不同文件即可

Epoch 1/100
252/252 [==============================] - 10s 33ms/step - loss: 1.1146 - acc: 0.0592
Epoch 2/100
252/252 [==============================] - 8s 33ms/step - loss: 1.0692 - acc: 0.0637
Epoch 3/100
252/252 [==============================] - 8s 32ms/step - loss: 1.0456 - acc: 0.0677
Epoch 4/100
252/252 [==============================] - 8s 33ms/step - loss: 1.0308 - acc: 0.0701
Epoch 5/100
252/252 [==============================] - 8s 33ms/step - loss: 1.0089 - acc: 0.0746
Epoch 6/100
252/252 [==============================] - 8s 32ms/step - loss: 0.9978 - acc: 0.0765
Epoch 7/100
252/252 [==============================] - 8s 32ms/step - loss: 1.0476 - acc: 0.0649
Epoch 8/100
252/252 [==============================] - 8s 33ms/step - loss: 0.9877 - acc: 0.0781
Epoch 9/100
252/252 [==============================] - 8s 33ms/step - loss: 0.9723 - acc: 0.0814
Epoch 10/100
252/252 [==============================] - 8s 33ms/step - loss: 0.9590 - acc: 0.0844
Epoch 11/100
252/252 [======

In [78]:
encoder_model=Model(encoder_inputs,encoder_states)
thought_input=[Input(shape=(num_neurons,)),Input(shape=(num_neurons,))]
decoder_outputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=thought_input)
decoder_states=[state_h,state_c]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=Model(inputs=[decoder_inputs]+thought_input,outputs=[decoder_outputs]+decoder_states)


In [91]:
import random
def sampple(preds,temperature=1.0):
    preds=np.array(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas=np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [121]:
def decode_sequence(input_seq):
    thought=encoder_model.predict(input_seq)
    target_seq=np.zeros((1,1,output_vocab_size))
    target_seq[0,0,target_token_index[stop_token]]=1
    stop_condition=False
    generated_sequence=''
    while not stop_condition:
        output_tokens,h,c=decoder_model.predict([target_seq]+thought)
        # print(output_tokens[0,-1,:])
        generated_token_idx=sampple(output_tokens[0,-1,:],temperature=0.3)#np.argmax(output_tokens[0,-1,:])
        generated_char=reverse_target_token_index[generated_token_idx]
        generated_sequence+=generated_char
        if (generated_char==stop_token or len(generated_sequence)>max_decoder_seq_length):
            stop_condition=True
        target_seq=np.zeros((1,1,output_vocab_size))
        target_seq[0,0,generated_token_idx]=1
        thought=[h,c]
    return generated_sequence


In [122]:
def response(input_text):
    input_seq=np.zeros((1,max_decoder_seq_length,input_vocab_size),dtype='float32')
    for i ,char in enumerate(input_text):
        input_seq[0,i,input_token_index[char]]=1
    decode_sentence=decode_sequence(input_seq)
    print('Bot reply:'+decode_sentence)


In [134]:
response("do you love girls?")

Bot reply:what is it and the some to the care in the one and here to should that the care.

